In [2]:
#! pip3 install tqdm
#! pip3 install tensorflow
#! pip install numpy
#! pip3 install pandas
#! pip3 install sklearn
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Subtract
from tensorflow.keras.callbacks import ModelCheckpoint, TerminateOnNaN, CSVLogger
tqdm.pandas()

MAX_LEN = 64

# Functions

In [3]:
def tokenize_inputs(phrase_a, phrase_b, tokenizer):

    tokenized_phrases = tokenizer.texts_to_sequences([phrase_a, phrase_b])

    # now loop through inputs and pad or reduce size if required
    tokenized_phrases_for_output = []
    for phrase in tokenized_phrases:
        if len(phrase) < MAX_LEN:
            length_to_pad = MAX_LEN - len(phrase)
            phrase_for_output = ([0] * length_to_pad) + phrase
        elif len(phrase) > MAX_LEN:
            phrase_for_output = phrase[-MAX_LEN:]
        else:
            phrase_for_output = phrase
        tokenized_phrases_for_output.append(phrase_for_output)

    return tf.constant(tokenized_phrases_for_output, dtype=tf.float64)

**Overview of model**

This is where we create the keras model. You will notice that I define a `common_lstm` layer. This is the siamese portion of the model which will have the same weights for both input (tokenized words). From here the output veotrs of these layers are subtracted before the resulting vector is passed through a series of dense layers which produces the fial classification.

In [4]:
def create_model():
  
    word_a_input_tokens = Input(
      shape=(MAX_LEN, 1), name='word_a_input_tokens'
      )
    word_b_input_tokens = Input(
      shape=(MAX_LEN, 1), name='word_b_input_tokens'
      )

    common_lstm = LSTM(64, return_sequences=False, activation="relu", name="common_lstm_layer")

    word_a_lstm_output = common_lstm(word_a_input_tokens)
    word_b_lstm_output = common_lstm(word_b_input_tokens)

    #concatenate_lstm_outputs
    concat_layer = Subtract(name="concatenate_lastm_outputs")(
      [word_a_lstm_output, word_b_lstm_output]
      )
    
    # dense layers before final classification
    dense_layers = Dense(64, activation="relu", name="first_dense_layer")(concat_layer)
    dense_layers = Dropout(0.5)(dense_layers)

    dense_layers = Dense(32, activation="relu", name="second_dense_layer")(dense_layers)
    dense_layers = Dropout(0.5)(dense_layers)

    dense_layers = Dense(8, activation="relu", name="third_dense_layer")(dense_layers)
    dense_layers = Dropout(0.5)(dense_layers)

    classification_layer = Dense(1, activation="sigmoid", name="classification_layer")(dense_layers)
    
    model = Model(
      inputs=[word_a_input_tokens, word_b_input_tokens], 
      outputs = classification_layer
      )

    model.compile(
      loss="binary_crossentropy",
      metrics=["accuracy"],
      optimizer="Adam"
    )

    return model

# Import data 

Load the data from the rhyme and non-rhyme files we created previously. Note due to hardware limitations I only focused on a subset of 1,000,000 word pairs but you can use more or less than this depending on your machine.

In [5]:
rhyme_df = pd.read_csv('data/rhymes/rhyme_df.csv')
rhyme_df = rhyme_df.dropna(subset=['word_a', 'word_b', 'rhyme'])
non_rhyme_df = pd.read_csv('data/rhymes/non_rhyme_df.csv')
non_rhyme_df = non_rhyme_df.dropna(subset=['word_a', 'word_b', 'rhyme'])

df = pd.concat([
        rhyme_df.sample(400_000, random_state=123), 
        non_rhyme_df.sample(400_000, random_state=123)
    ])
del rhyme_df, non_rhyme_df

df.head()

,rhyme_id,rhyme_group_id,word_a,word_b,rhyme
190131,246172,111,es his,esse is,1
32885,32886,9,chaebol,grable,1
206073,273379,125,braithwaite of,distillate of,1
356273,480878,207,spry,magpie,1
337155,461757,202,geez,sees,1


# Tokenize inputs

Create and fit a tokenizer on the dataset.

In [6]:
tokenizer = Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(df['word_a'] + df['word_b'])

df['word_tokens'] = df.apply(lambda row: tokenize_inputs(row['word_a'], row['word_b'], tokenizer), axis=1)

# Split data into train (60%) test (30%) and validation (10%) sets

Use stratified random sampling to create train, test and va;idation datasets.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    list(df['word_tokens']), list(df['rhyme']), stratify=df['rhyme'], 
    test_size=0.4, random_state=123
    )
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.25, random_state=123
    )

In [10]:
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_val = tf.convert_to_tensor(X_val)
y_val = tf.convert_to_tensor(y_val)

# Train model

Call our model function and fit it on our training data

In [10]:
model = create_model()

model_checkpoint = ModelCheckpoint("models/rhyme_model.hdf5",monitor="val_loss")
terminate_on_nan = TerminateOnNaN()
csv_logger = CSVLogger('training.log')

history = model.fit(
    [X_train[:, 0], X_train[:, 1]],
    y_train,
    batch_size=128,
    epochs=100,
    callbacks=[model_checkpoint, terminate_on_nan, csv_logger],
    validation_data=([X_val[:, 0], X_val[:, 1]], y_val)
)


Epoch 1/100
3750/3750 [==============================] - 164s 43ms/step - loss: 0.4662 - accuracy: 0.8006 - val_loss: 0.3033 - val_accuracy: 0.8729
Epoch 2/100
3750/3750 [==============================] - 181s 48ms/step - loss: 0.3586 - accuracy: 0.8581 - val_loss: 0.2489 - val_accuracy: 0.9097
Epoch 3/100
3750/3750 [==============================] - 173s 46ms/step - loss: 0.3225 - accuracy: 0.8757 - val_loss: 0.2138 - val_accuracy: 0.9248
Epoch 4/100
3750/3750 [==============================] - 174s 46ms/step - loss: 0.2979 - accuracy: 0.8868 - val_loss: 0.2082 - val_accuracy: 0.9198
Epoch 5/100
3750/3750 [==============================] - 177s 47ms/step - loss: 0.2811 - accuracy: 0.8942 - val_loss: 0.1881 - val_accuracy: 0.9333
Epoch 6/100
3750/3750 [==============================] - 176s 47ms/step - loss: 0.2723 - accuracy: 0.8988 - val_loss: 0.1750 - val_accuracy: 0.9379
Epoch 7/100
3750/3750 [==============================] - 177s 47ms/step - loss: 0.2592 - accuracy: 0.9034 - val_

# Test model

In [11]:
# load the model
model = load_model("models/rhyme_model.hdf5")

X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
y_pred = y_pred > 0.5


7500/7500 [==============================] - 53s 7ms/step


In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98    120000
           1       0.97      0.99      0.98    120000

    accuracy                           0.98    240000
   macro avg       0.98      0.98      0.98    240000
weighted avg       0.98      0.98      0.98    240000



# Run model on some examples 

In [13]:
samples = [
   ["soap", "hope"], # 
   ["cat", "hat"], # 
   ["sliver", "cleaver"],#
   ["inspire", "desire"], #
   ["tomato", "salad"], #
]

sample_tokens = [tokenize_inputs(lyrics[0], lyrics[1], tokenizer) for lyrics in samples]
sample_tokens = tf.convert_to_tensor(sample_tokens)
sample_pred = model.predict([sample_tokens[:, 0], sample_tokens[:, 1]])
predictions = [round(pred[0], 4) for pred in sample_pred]
for i in range(len(samples)):
    print(f"Lyric 1: {samples[i][0]}")
    print(f"Lyric 2: {samples[i][1]}")
    print(f"{'Rhyme' if predictions[i] > 0.5 else 'Non-rhyme'}({predictions[i]})")
    print("---------------\n")

1/1 [==============================] - 0s 420ms/step
Lyric 1: soap
Lyric 2: hope
Rhyme(0.6202999949455261)
---------------

Lyric 1: cat
Lyric 2: hat
Rhyme(0.9830999970436096)
---------------

Lyric 1: sliver
Lyric 2: cleaver
Rhyme(0.9830999970436096)
---------------

Lyric 1: inspire
Lyric 2: desire
Rhyme(0.9830999970436096)
---------------

Lyric 1: tomato
Lyric 2: salad
Non-rhyme(0.0003000000142492354)
---------------



In [19]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
cm = confusion_matrix(y_val, x_val)
df_cm = pd.DataFrame(cm, index = [0,1],
              columns = [0,1])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,cmap="RdPu")
plt.savefig('confusion_matrix.png',bbox_inches = 'tight')

ValueError: Found input variables with inconsistent numbers of samples: [80000, 240000]